In [1]:
print("Name : SANSKAAR CHAUHAN")

Name : SANSKAAR CHAUHAN


In [2]:
# File 1: Data Loading and Cleaning

import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load datasets
print("Loading datasets...")

sentiment_df = pd.read_csv('fear_greed_index.csv')
trades_df = pd.read_csv('historical_data.csv')

print(f"Sentiment data: {len(sentiment_df)} records")
print(f"Trading data: {len(trades_df)} records")
print(f"Unique traders: {trades_df['Account'].nunique()}")
print(f"Unique coins: {trades_df['Coin'].nunique()}")

Loading datasets...
Sentiment data: 2644 records
Trading data: 211224 records
Unique traders: 32
Unique coins: 246


In [4]:
# Clean sentiment data
print("\nCleaning sentiment data...")

sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
sentiment_df = sentiment_df.rename(columns={
    'value': 'fear_greed_value',
    'classification': 'sentiment_category'
})


Cleaning sentiment data...


In [5]:
# Add time features
sentiment_df['year'] = sentiment_df['date'].dt.year
sentiment_df['month'] = sentiment_df['date'].dt.month
sentiment_df['day_of_week'] = sentiment_df['date'].dt.dayofweek

In [6]:
# Numeric encoding for sentiment
sentiment_mapping = {
    'Extreme Fear': 1,
    'Fear': 2,
    'Neutral': 3,
    'Greed': 4,
    'Extreme Greed': 5
}
sentiment_df['sentiment_numeric'] = sentiment_df['sentiment_category'].map(sentiment_mapping)

In [7]:
# Binary flags
sentiment_df['is_fear'] = sentiment_df['sentiment_category'].isin(['Extreme Fear', 'Fear'])
sentiment_df['is_greed'] = sentiment_df['sentiment_category'].isin(['Greed', 'Extreme Greed'])

print("Sentiment data cleaned")
print(sentiment_df.head())

Sentiment data cleaned
    timestamp  fear_greed_value sentiment_category       date  year  month  \
0  1517463000                30               Fear 2018-02-01  2018      2   
1  1517549400                15       Extreme Fear 2018-02-02  2018      2   
2  1517635800                40               Fear 2018-02-03  2018      2   
3  1517722200                24       Extreme Fear 2018-02-04  2018      2   
4  1517808600                11       Extreme Fear 2018-02-05  2018      2   

   day_of_week  sentiment_numeric  is_fear  is_greed  
0            3                  2     True     False  
1            4                  1     True     False  
2            5                  2     True     False  
3            6                  1     True     False  
4            0                  1     True     False  


In [8]:
# Clean trading data
print("\nCleaning trading data...")

trades_df['trade_datetime'] = pd.to_datetime(trades_df['Timestamp IST'], format='%d-%m-%Y %H:%M')
trades_df['trade_date'] = pd.to_datetime(trades_df['trade_datetime'].dt.date)


Cleaning trading data...


In [9]:
# Add time features
trades_df['year'] = trades_df['trade_datetime'].dt.year
trades_df['month'] = trades_df['trade_datetime'].dt.month
trades_df['hour'] = trades_df['trade_datetime'].dt.hour
trades_df['day_of_week'] = trades_df['trade_datetime'].dt.dayofweek

In [10]:
# Clean column names
trades_df.columns = trades_df.columns.str.replace(' ', '_')

In [11]:
# Trade features
trades_df['is_profitable'] = trades_df['Closed_PnL'] > 0
trades_df['is_loss'] = trades_df['Closed_PnL'] < 0
trades_df['is_closed'] = trades_df['Closed_PnL'] != 0

trades_df['is_long'] = trades_df['Direction'].str.contains('Long', na=False)
trades_df['is_short'] = trades_df['Direction'].str.contains('Short', na=False)

In [12]:
# Return percentage
trades_df['return_pct'] = np.where(
    trades_df['is_closed'] & (trades_df['Size_USD'] > 0),
    (trades_df['Closed_PnL'] / trades_df['Size_USD']) * 100,
    0
)

In [13]:
# Trade size categories
trades_df['trade_size_category'] = pd.cut(
    trades_df['Size_USD'],
    bins=[0, 100, 500, 2000, 10000, np.inf],
    labels=['Micro', 'Small', 'Medium', 'Large', 'Very Large']
)

print("Trading data cleaned")
print(trades_df[['Account', 'Coin', 'Side', 'Size_USD', 'Closed_PnL', 'trade_date']].head())

Trading data cleaned
                                      Account  Coin Side  Size_USD  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY   7872.16   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY    127.68   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY   1150.63   
3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY   1142.04   
4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107  BUY     69.75   

   Closed_PnL trade_date  
0         0.0 2024-12-02  
1         0.0 2024-12-02  
2         0.0 2024-12-02  
3         0.0 2024-12-02  
4         0.0 2024-12-02  


In [14]:
# Merge datasets
print("\nMerging datasets...")

merged_df = trades_df.merge(
    sentiment_df[['date', 'fear_greed_value', 'sentiment_category', 
                  'sentiment_numeric', 'is_fear', 'is_greed']],
    left_on='trade_date',
    right_on='date',
    how='left'
)

print(f"Merged data: {len(merged_df)} records")
print(f"Records with sentiment: {merged_df['sentiment_category'].notna().sum()}")


Merging datasets...
Merged data: 211224 records
Records with sentiment: 211218


In [15]:
# Create daily aggregates
print("\nCreating daily aggregates...")

daily_stats = merged_df.groupby(['Account', 'trade_date']).agg({
    'Closed_PnL': ['sum', 'mean', 'count'],
    'Size_USD': ['sum', 'mean'],
    'is_profitable': 'sum',
    'is_closed': 'sum',
    'Fee': 'sum'
}).reset_index()

daily_stats.columns = ['Account', 'date', 'total_pnl', 'avg_pnl', 'num_trades',
                       'total_volume', 'avg_trade_size', 'num_profitable', 
                       'num_closed', 'total_fees']

daily_stats['win_rate'] = np.where(
    daily_stats['num_closed'] > 0,
    (daily_stats['num_profitable'] / daily_stats['num_closed']) * 100,
    0
)

daily_stats['net_pnl'] = daily_stats['total_pnl'] - daily_stats['total_fees']


Creating daily aggregates...


In [16]:
# Merge sentiment with daily stats
daily_stats = daily_stats.merge(
    sentiment_df[['date', 'fear_greed_value', 'sentiment_category', 'sentiment_numeric']],
    on='date',
    how='left'
)

print("Daily aggregates created")
print(daily_stats.head())

Daily aggregates created
                                      Account       date  total_pnl  \
0  0x083384f897ee0f19899168e3b1bec365f52a9012 2024-11-11        0.0   
1  0x083384f897ee0f19899168e3b1bec365f52a9012 2024-11-17        0.0   
2  0x083384f897ee0f19899168e3b1bec365f52a9012 2024-11-18        0.0   
3  0x083384f897ee0f19899168e3b1bec365f52a9012 2024-11-22   -21227.0   
4  0x083384f897ee0f19899168e3b1bec365f52a9012 2024-11-26     1603.1   

       avg_pnl  num_trades  total_volume  avg_trade_size  num_profitable  \
0     0.000000         177     900880.13     5089.718249               0   
1     0.000000          68     542413.18     7976.664412               0   
2     0.000000          40     949380.00    23734.500000               0   
3 -1768.916667          12     338240.00    28186.666667               0   
4    59.374074          27     465700.00    17248.148148              12   

   num_closed  total_fees  win_rate       net_pnl  fear_greed_value  \
0           0  167.7

In [17]:
# Create trader-level stats
print("\nCreating trader performance summary...")

trader_stats = merged_df.groupby('Account').agg({
    'Closed_PnL': ['sum', 'mean', 'std'],
    'Size_USD': ['sum', 'mean'],
    'is_profitable': 'sum',
    'is_closed': 'sum',
    'Fee': 'sum',
    'trade_date': ['min', 'max']
}).reset_index()

trader_stats.columns = ['Account', 'total_pnl', 'avg_pnl', 'pnl_std',
                        'total_volume', 'avg_trade_size', 'num_wins',
                        'num_closed_trades', 'total_fees', 'first_trade', 'last_trade']

trader_stats['win_rate'] = np.where(
    trader_stats['num_closed_trades'] > 0,
    (trader_stats['num_wins'] / trader_stats['num_closed_trades']) * 100,
    0
)

trader_stats['net_pnl'] = trader_stats['total_pnl'] - trader_stats['total_fees']
trader_stats['trading_days'] = (trader_stats['last_trade'] - trader_stats['first_trade']).dt.days

trader_stats = trader_stats.sort_values('net_pnl', ascending=False)

print("Trader stats created")
print(trader_stats.head(10))


Creating trader performance summary...
Trader stats created
                                       Account     total_pnl     avg_pnl  \
27  0xb1231a4a2dd02f2276fa3c5e2a2f3436e6bfed23  2.143383e+06  145.481748   
0   0x083384f897ee0f19899168e3b1bec365f52a9012  1.600230e+06  419.127768   
29  0xbaaaf6571ab7d571043ff1e313a9609a10637864  9.401638e+05   44.364091   
31  0xbee1707d6b44d4d52bfe19e41f8a828645437aab  8.360806e+05   20.806305   
13  0x513b8629fe877bb581bf244e326a047b249c4ff1  8.404226e+05   68.684419   
11  0x4acb90e786d897ecffb614dc822eb231b4ffb9f4  6.777471e+05  155.589314   
15  0x72743ae2822edd658c0c50608fd7c5c501b2afbd  4.293556e+05  270.034947   
9   0x430f09841d65beb3f27765503d0f850b8bce7713  4.165419e+05  336.735548   
16  0x72c6a4624e1dffa724e6d00d64ceae698af892a0  4.030115e+05  281.826227   
17  0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4  3.790954e+05   38.319560   

        pnl_std  total_volume  avg_trade_size  num_wins  num_closed_trades  \
27  1374.026402  5.65435

In [18]:
# Export cleaned datasets
print("\nExporting cleaned data...")

try:
    sentiment_df.to_csv('cleaned_sentiment.csv', index=False)
    print("Exported: cleaned_sentiment.csv")
except Exception as e:
    print(f"Error exporting sentiment: {e}")

try:
    merged_df.to_csv('cleaned_merged_trades.csv', index=False)
    print("Exported: cleaned_merged_trades.csv")
except Exception as e:
    print(f"Error exporting merged trades: {e}")

try:
    daily_stats.to_csv('daily_trader_stats.csv', index=False)
    print("Exported: daily_trader_stats.csv")
except Exception as e:
    print(f"Error exporting daily stats: {e}")

try:
    trader_stats.to_csv('trader_performance_summary.csv', index=False)
    print("Exported: trader_performance_summary.csv")
except Exception as e:
    print(f"Error exporting trader stats: {e}")

print("\nData cleaning complete!")
print("Check your directory for the 4 CSV files.")


Exporting cleaned data...
Exported: cleaned_sentiment.csv
Exported: cleaned_merged_trades.csv
Exported: daily_trader_stats.csv
Exported: trader_performance_summary.csv

Data cleaning complete!
Check your directory for the 4 CSV files.
